In [ ]:
# Montar Drive e imports
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import cv2
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm

In [ ]:

# Config diretórios
BASE_DIR = Path('/content/drive/MyDrive/UTP_ATM_V3')

DIR_A_ORIGINAL = BASE_DIR / 'A_ORIGINAL'
DIR_N_ORIGINAL = BASE_DIR / 'N_ORIGINAL'

DIR_A_PADRON = BASE_DIR / 'A_PADRONIZADO'
DIR_N_PADRON = BASE_DIR / 'N_PADRONIZADO'

# Criar pastas de saída se não existirem
for d in [DIR_A_PADRON, DIR_N_PADRON]:
    d.mkdir(parents=True, exist_ok=True)

VALID_EXTS = {'.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff'}
DRY_RUN = False

LOG_FILE = BASE_DIR / 'rastreio_processamento.txt'


Mounted at /content/drive


In [ ]:
# detecção de borda + leitura
def detect_right_white_border(
    img_gray,
    bright_quantile=0.985,
    min_run_px=3,
    min_frac_rows=0.35,
    exclude_margin_ratio=0.05,
    diff_px=2
):
    """Retorna True se há “borda branca” predominante na direita."""
    H, W = img_gray.shape[:2]
    top = int(H * exclude_margin_ratio)
    bottom = int(H * (1 - exclude_margin_ratio))
    roi = img_gray[top:bottom, :]

    thr = int(min(254, max(200, np.quantile(roi, bright_quantile))))

    right_runs, left_runs = [], []

    for row in roi:
        run_r = run_l = 0

        # direita -> esquerda
        for c in range(W - 1, -1, -1):
            if row[c] >= thr:
                run_r += 1
            else:
                break

        # esquerda -> direita
        for c in range(W):
            if row[c] >= thr:
                run_l += 1
            else:
                break

        right_runs.append(run_r)
        left_runs.append(run_l)

    right_med = float(np.median(right_runs))
    left_med  = float(np.median(left_runs))

    frac_rows = sum(1 for x in right_runs if x >= min_run_px) / max(1, len(right_runs))

    decision = (frac_rows >= min_frac_rows) and (right_med >= left_med + diff_px)
    return decision


def read_image_gray(path: Path):
    img = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f'Falha ao ler: {path}')
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    return clahe.apply(img)


In [ ]:
def padronizar_espelhar(src_folder: Path, dst_folder: Path):
    """
    Lê imagens já renomeadas (img1A, img2N, ...)
    e grava em dst_folder como img1A_padronizado, etc.
    Espelha se a borda branca predominante estiver à direita.
    """
    imgs = [p for p in src_folder.glob('img*') if p.suffix.lower() in VALID_EXTS]
    imgs.sort()
    kept = flipped = err = 0

    for p in tqdm(imgs, desc=f"Padronizando {src_folder.name}"):
        try:
            g = read_image_gray(p)
            do_flip = detect_right_white_border(g)

            img = cv2.imread(str(p), cv2.IMREAD_UNCHANGED)
            if img is None:
                raise ValueError("falha ao ler imagem colorida")

            out_path = dst_folder / f"{p.stem}_padronizado{p.suffix.lower()}"

            if do_flip:
                img_proc = cv2.flip(img, 1)
                flipped += 1
                action = "padronizada (espelhada)"
            else:
                img_proc = img
                kept += 1
                action = "padronizada (mantida)"

            if not DRY_RUN:
                cv2.imwrite(str(out_path), img_proc)

            append_log(f"{p.stem} -> {action} => {out_path.relative_to(BASE_DIR)}")

        except Exception as e:
            err += 1
            append_log(f"{p.name} -> ERRO na padronização: {e}")

    print(f"\n[{dst_folder.name}] ✅ Padronização concluída")
    print(f"  • Total: {len(imgs)} | Mantidas: {kept} | Espelhadas: {flipped} | Erros: {err}")


In [ ]:
def append_log(line: str):
    """Acrescenta uma linha ao TXT de rastreio (sem data)."""
    with open(LOG_FILE, 'a', encoding='utf-8') as f:
        prefix = "[SIMULACAO] " if DRY_RUN else ""
        f.write(prefix + line.rstrip() + '\n')

In [ ]:
# Execução
assert DIR_A_ORIGINAL.exists(), f"Não encontrei {DIR_A_ORIGINAL}"
assert DIR_N_ORIGINAL.exists(), f"Não encontrei {DIR_N_ORIGINAL}"

padronizar_espelhar(DIR_A_ORIGINAL, DIR_A_PADRON)
padronizar_espelhar(DIR_N_ORIGINAL, DIR_N_PADRON)

print("\n✨ Etapa de PADRONIZAÇÃO finalizada.")
print("📄 Rastreio em:", LOG_FILE)


Padronizando A_ORIGINAL:   0%|          | 0/100 [00:00<?, ?it/s]


[A_PADRONIZADO] ✅ Padronização concluída
  • Total: 100 | Mantidas: 53 | Espelhadas: 47 | Erros: 0


Padronizando N_ORIGINAL:   0%|          | 0/100 [00:00<?, ?it/s]


[N_PADRONIZADO] ✅ Padronização concluída
  • Total: 100 | Mantidas: 55 | Espelhadas: 45 | Erros: 0

✨ Etapa de PADRONIZAÇÃO finalizada.
📄 Rastreio em: /content/drive/MyDrive/UTP_ATM_V3/rastreio_processamento.txt
